In [1]:
#there are some outliers among the parent-child samples on plots from pc-relate. Investigate these and create a list of families to look at.
import hail as hl
import pyspark

sc = pyspark.SparkContext()
tmp_dir = "hdfs://spark-master:9820/"
lustre_dir = "file:///lustre/scratch123/qc/"
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference="GRCh38")

2022-06-15 08:49:17 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.1.2
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.88-7d39379870da
LOGGING: writing to /home/ubuntu/jupyter/hail-20220615-0849-0.2.88-7d39379870da.log


In [2]:
htfile = lustre_dir + "matrixtables/pcrelate_annotated.ht"
ht = hl.read_table(htfile)
ht.show()

,,,,,,,
i,j,,,,,,
s,s,kin,ibd0,ibd1,ibd2,pair,relationship
str,str,float64,float64,float64,float64,str,str
"""EGAN00003143345""","""EGAN00003143360""",3.94e-03,1.03e+00,-7.14e-02,4.36e-02,"""EGAN00003143345_EGAN00003143360""","""unrelated"""
"""EGAN00003143345""","""EGAN00003143411""",6.22e-03,1.02e+00,-6.29e-02,4.39e-02,"""EGAN00003143345_EGAN00003143411""","""unrelated"""
"""EGAN00003143345""","""EGAN00003143419""",8.82e-04,1.04e+00,-8.10e-02,4.23e-02,"""EGAN00003143345_EGAN00003143419""","""unrelated"""
"""EGAN00003143345""","""EGAN00003143420""",6.98e-03,1.01e+00,-4.51e-02,3.65e-02,"""EGAN00003143345_EGAN00003143420""","""unrelated"""
"""EGAN00003143345""","""EGAN00003143422""",-1.50e-03,1.04e+00,-7.75e-02,3.57e-02,"""EGAN00003143345_EGAN00003143422""","""unrelated"""
"""EGAN00003143345""","""EGAN00003143434""",-2.13e-04,1.07e+00,-1.34e-01,6.67e-02,"""EGAN00003143345_EGAN00003143434""","""unrelated"""
"""EGAN00003143345""","""EGAN00003143435""",-3.92e-03,1.06e+00,-1.00e-01,4.22e-02,"""EGAN00003143345_EGAN00003143435""","""unrelated"""


In [3]:
#select those which are just parent-child and plot kin vs IBD0 - the original plot didn't show all points so want to check there is a suitable cut off that can be used.
parent_child_ht = ht.filter(ht.relationship == 'parent_child')

from hail.plot import show
hl.plot.output_notebook()

p = hl.plot.scatter(parent_child_ht.ibd0, parent_child_ht.kin, label = parent_child_ht.relationship, xlabel='IBD0', ylabel='kin', collect_all = True)
show(p)


Loading BokehJS ...

In [4]:
#unrelated "parent-child" pairs clearly seperated from the related ones. kin < 0.12 and IBD0 > 0.55. A threshold of 0.15 for kin is going to work here so create table of just unrelated "parent-child" pairs
unrelated_parents_ht = parent_child_ht.filter(parent_child_ht.kin < 0.15)
unrelated_parents_ht.count()

62

In [5]:
#get the pair ids of these into a list
unrelated_pairs = unrelated_parents_ht.pair.collect()

In [9]:
#load in trio data and map to pairs
pedfile = "/lustre/scratch123/qc/resources/trios.ped"
pair_trio = {}
with open (pedfile, 'r') as f:
    lines = f.readlines()
    for l in lines:
        ldata = l.split()
        fam = ldata[0]
        proband = ldata[1]
        dad = ldata[2]
        mum = ldata[3]
        proband_dad = proband + "_" + dad
        proband_mum = proband + "_" + mum
        pair_trio[proband_dad] = {}
        pair_trio[proband_dad]['fam'] = fam
        pair_trio[proband_dad]['relationship'] = 'proband_father'
        pair_trio[proband_mum] = {}
        pair_trio[proband_mum]['fam'] = fam 
        pair_trio[proband_mum]['relationship'] = 'proband_mother'
        #also add opposite orientation
        dad_proband = dad + "_" + proband
        mum_proband = mum + "_" + proband
        pair_trio[dad_proband] = {}
        pair_trio[dad_proband]['fam'] = fam
        pair_trio[dad_proband]['relationship'] = 'proband_father'
        pair_trio[mum_proband] = {}
        pair_trio[mum_proband]['fam'] = fam 
        pair_trio[mum_proband]['relationship'] = 'proband_mother'
        
        

In [10]:
#for each pair in unrelated parents, identify family id and relationship between the two individuals
outdata = []
header = ("\t").join(['family', 'relationship'])
outdata.append(header)
for p in unrelated_pairs:
    line = ("\t").join([pair_trio[p]['fam'], pair_trio[p]['relationship']])
    outdata.append(line)
    
outfile = "/lustre/scratch123/qc/annotations/unrelated_parents.txt"
with open(outfile, 'w') as o:
    o.write(("\n").join(outdata))

In [11]:
#there are two points which don't look like parent-child but still seem possibly related - have a look at these
distant_rels = unrelated_parents_ht.filter(unrelated_parents_ht.kin > 0.1)
distant_rels.show()

,,,,,,,
i,j,,,,,,
s,s,kin,ibd0,ibd1,ibd2,pair,relationship
str,str,float64,float64,float64,float64,str,str
"""EGAN00003144865""","""EGAN00003308293""",1.03e-01,6.29e-01,3.31e-01,4.00e-02,"""EGAN00003144865_EGAN00003308293""","""parent_child"""
"""EGAN00003147085""","""EGAN00003309123""",1.10e-01,5.93e-01,3.75e-01,3.28e-02,"""EGAN00003147085_EGAN00003309123""","""parent_child"""


In [12]:
#do the unrelated parent-child pairs roughly correspond to those families with high mendelain errors?
mtfile = lustre_dir + "matrixtables/mt_varqc_splitmulti.mt"
mt = hl.read_matrix_table(mtfile)
mtf = mt.filter_rows((mt.info.QD >= 2) & (mt.info.FS <= 60) & (mt.info.MQ >= 30), keep = True ) #apply haird filters

In [13]:
pedfile = lustre_dir + "resources/trios.ped"
ped = hl.Pedigree.read(pedfile)

In [18]:
all_errors, per_fam, per_sample, per_variant = hl.mendel_errors(mtf['GT'], ped)

In [19]:
per_fam_err = per_fam.filter(per_fam.errors >10000, keep = True)#keep only those with excessive errors
per_fam_err.show()

2022-06-15 12:59:19 Hail: INFO: Ordering unsorted dataset with network shuffle5]


,,,,,
pat_id,mat_id,fam_id,children,errors,snp_errors
str,str,str,int32,int64,int64
"""EGAN00003234865""","""EGAN00003237377""","""19480A""",1,19883,16587
"""EGAN00003234992""","""EGAN00003237433""","""1091A""",1,19575,16399
"""EGAN00003235166""","""EGAN00003306921""","""8200B""",2,12696,6267
"""EGAN00003235291""","""EGAN00003307158""","""2088A""",1,19431,16180
"""EGAN00003235299""","""EGAN00003237523""","""16088A""",1,24279,19699
"""EGAN00003235321""","""EGAN00003236747""","""6258A""",1,23103,18414
"""EGAN00003235338""","""EGAN00003237713""","""7151A""",1,20459,17092
"""EGAN00003235428""","""EGAN00003237486""","""4136A""",1,48867,41887


In [20]:
per_fam_err.count()

2022-06-15 13:02:50 Hail: INFO: Ordering unsorted dataset with network shuffle5]


82

In [22]:
dad_errs = per_fam_err.pat_id.collect()
mum_errs = per_fam_err.mat_id.collect()
parent_errs = mum_errs + dad_errs
unrelated_high_mendel = unrelated_parents_ht.filter( (hl.set(parent_errs).contains(unrelated_parents_ht.i.s)) | (hl.set(parent_errs).contains(unrelated_parents_ht.j.s)), keep = True)
unrelated_high_mendel.count()

2022-06-15 13:24:23 Hail: INFO: Ordering unsorted dataset with network shuffle  
2022-06-15 13:26:34 Hail: INFO: Ordering unsorted dataset with network shuffle5]


59